# Testing notebook for scikit-BOLD code
This notebook offers some code to test the functionality of the scikit-BOLD repository.

In [3]:
from __future__ import print_function, division, absolute_import
import numpy as np
import nibabel as nib
import glob
import os
import scikit_bold.data2mvp.glm2mvp as glm
import scikit_bold.transformers.transformers as trans
import scikit_bold.utils.mvp_utils as utils
from scikit_bold.utils.mvp_utils import MvpResults
import glob
import os
import warnings
import numpy as np
from os.path import join as opj
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from scikit_bold.data2mvp.glm2mvp import Fsl2mvp
from scikit_bold.utils.mvp_utils import MvpResults, MvpAverageResults, DataHandler
from scikit_bold.transformers.transformers import *

home_dir = '/home/lukas/Dropbox/PhD_projects/scikit-bold/scikit_bold/'

### Converting first-level GLM to scikit-learn compatible format

In [ ]:
testdirs = glob.glob(os.path.join(home_dir, 'tests', 'HWW_00?', '*.feat'))
mask = '/home/lukas/Dropbox/PhD_projects/DynamicAffect_Multiscale/ROIs/GrayMatter.nii.gz'

for d in testdirs:
    test = glm.Fsl2mvp(d, mask_threshold=10, beta2tstat=True, ref_space='epi', mask_path=mask, remove_class=[])
    test.glm2mvp_and_merge()

### Testing pipelines
First, we'll load in the data.

In [21]:
project_dir = '/home/lukas/Dropbox/PhD_projects/scikit-bold/scikit_bold/tests/'

# Turn-off UserWarning to avoid clutter (occurs in univariate feature selection)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

#  Definition of data-dirs
paths = glob.glob(os.path.join(project_dir, 'HWW_00?'))

### Analysis parameters ###
iterations = 10
n_test = 1
zvalue = 2.5
score_method = 'voting'

scaler = StandardScaler()
transformer = MeanEuclidean(cutoff=zvalue, normalize=True)
clf = SVC(kernel='linear', probability=True)
pipeline = Pipeline([('euclidean', transformer), ('scaler', scaler), ('classifier', clf)])
gs_params = dict(euclidean__cutoff=np.linspace(1.5, 2.3, 3))
pipeline = GridSearchCV(pipeline, param_grid=gs_params, n_jobs=-1)

### Loop over subjects ###
for i in range(len(paths)):

    sub_name = os.path.basename(paths[i])
    print('\nProcessing %s' % sub_name)

    # Loading
    data = utils.DataHandler(identifier='merged').load_separate_sub(paths[i])
    data.n_class = np.unique(data.y).size
    folds = StratifiedShuffleSplit(data.y, n_iter=iterations,
                                        test_size=n_test * data.n_class)

    results = MvpResults(data, iterations, method=score_method, feature_scoring='accuracy')

    for j, folds in enumerate(folds):

        train_idx, test_idx = folds
        X_train, X_test = data.X[train_idx, :], data.X[test_idx, :]
        y_train, y_test = data.y[train_idx], data.y[test_idx]
        
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict_proba(X_test)
        results.update_results(test_idx=test_idx, y_pred=y_pred)
        
    # Compute average performance and write results (.csv)
    results.compute_score().write_results(project_dir, convert2mni=True)
    
# Initialize 'averager' and write results
averager = MvpAverageResults(project_dir, '', params=None, cleanup=False)
averager.average()

print('Analysis finished.')

""" END OF ANALYSIS """



Processing HWW_003
Accuracy over iterations: 0.896552


AttributeError: 'NoneType' object has no attribute 'write_results'

True